In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,TrainValidationSplit

In [0]:
spark = SparkSession.builder.appName("Colloborative-Recommendor-System").getOrCreate()

In [0]:
path = "/FileStore/tables/movies.csv"

df = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("path", path) \
  .load()

In [0]:
from collections import Counter

movies = df.select('movieId').rdd.flatMap(lambda x: x).collect()
counts = Counter(movies)
print('Top 10 movies')
print(sorted(counts, key=counts.get, reverse=True)[0:10])

Top 10 movies
[29, 50, 51, 6, 22, 2, 15, 55, 68, 94]

In [0]:
users = df.select('userId').rdd.flatMap(lambda x: x).collect()

counts = Counter(users)
print('Top 10 users')
print(sorted(counts, key=counts.get, reverse=True)[0:10])

Top 10 users
[6, 14, 11, 22, 4, 12, 7, 9, 18, 23]

In [0]:
als = ALS(maxIter=20, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

In [0]:
# Model with 80/20 split
(training80, test20) = df.randomSplit([0.8, 0.2])
modelA = als.fit(training)
predictionsA = model.transform(test)
predictionsA.show(1)

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1| 0| 1.3261495|
+-------+------+------+----------+
only showing top 1 row

In [0]:
# Model with 60/40 split
(training60, test40) = df.randomSplit([0.6, 0.4])
modelB = als.fit(training)
predictionsB = model.transform(test)
predictionsB.show(1)

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1| 0| 1.3261495|
+-------+------+------+----------+
only showing top 1 row

In [0]:
evaluatorrmse = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluatorrmse.evaluate(predictionsA)
print("RMSE for Model with 80/20 split:", rmse)
rmse = evaluatorrmse.evaluate(predictionsB)
print("RMSE for Model with 60/40 split:", rmse)

RMSE for Model with 80/20 split: 0.4866851446765971
RMSE for Model with 60/40 split: 0.554410771942163

In [0]:
evaluatormse = RegressionEvaluator(metricName='mse', labelCol='rating', predictionCol='prediction')
mse = evaluatormse.evaluate(predictionsA)
print("MSE for Model with 80/20 split:", mse)
mse = evaluatormse.evaluate(predictionsB)
print("MSE for Model with 60/40 split:", mse)

MSE for Model with 80/20 split: 0.23686243004888027
MSE for Model with 60/40 split: 0.30737130404550517

In [0]:
evaluatormae = RegressionEvaluator(metricName='mae', labelCol='rating', predictionCol='prediction')
mae = evaluatormae.evaluate(predictionsA)
print("MAE for Model with 80/20 split:", mae)
mae = evaluatormae.evaluate(predictionsB)
print("MAE for Model with 60/40 split:", mae)

MAE for Model with 80/20 split: 0.3243356847337314
MAE for Model with 60/40 split: 0.34710336310022016

In [0]:
parameters = ParamGridBuilder().addGrid(als.regParam, [0.01, 0.05]).addGrid(als.rank, [10, 50]).build()

trainvs = TrainValidationSplit(estimator=als, estimatorParamMaps=parameters, evaluator=evaluatorrmse)

In [0]:
model = trainvs.fit(training80)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
predictionsA = modelA.transform(test20)

rmse = evaluatorrmse.evaluate(predictions)

In [0]:
predictions.show(1)

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 4| 12| 0.8869905|
+-------+------+------+----------+
only showing top 1 row

In [0]:
print("RMSE value for model with 60/40 split after hyperparameter tuning:", rmse)

RMSE value for model with 60/40 split after hyperparameter tuning: 1.0527589276233482

In [0]:
def recommendmovies(user):
  user_subset = df.where(df.userId == user)
  user_recs = model.recommendForUserSubset(user_subset, 75)
  recs = (user_recs.select("userId", explode("recommendations").alias("recommendation"))\
          .select("userId", "recommendation.*")).sort('rating', ascending=False)
  recs.select('userId','movieId','rating')
  top_15_recs = (recs.join(user_subset, on=['movieId'], how='left_anti')).show(15)
  return top_15_recs

In [0]:
recommendmovies(11)

+-------+------+---------+
movieId|userId| rating|
+-------+------+---------+
 8| 11|3.7538939|
 65| 11|3.2291427|
 55| 11|3.2278914|
 49| 11| 3.071352|
 34| 11|2.7378283|
 33| 11|2.6465523|
 83| 11|2.5463712|
 7| 11|2.2645674|
 74| 11|2.2168248|
 87| 11|2.1206365|
 73| 11| 2.087879|
 17| 11|2.0305462|
 46| 11|1.9102235|
 96| 11|1.8122079|
 68| 11| 1.733347|
+-------+------+---------+
only showing top 15 rows

In [0]:
recommendmovies(23)

+-------+------+---------+
movieId|userId| rating|
+-------+------+---------+
 90| 23|4.8699217|
 17| 23|4.3150325|
 46| 23|4.1669397|
 35| 23| 4.08302|
 20| 23|3.2640066|
 79| 23|2.9868133|
 94| 23|2.8531518|
 31| 23|2.6271446|
 16| 23|2.6233954|
 81| 23| 2.594754|
 7| 23|2.5360503|
 91| 23|2.4784243|
 56| 23| 2.395237|
 19| 23|2.2295318|
 9| 23|1.9445672|
+-------+------+---------+
only showing top 15 rows